## 面试题35：复杂链表的复制

题目：输入一个复杂链表（每个节点中有节点值，以及两个指针，一个指向下一个节点，另一个特殊指针指向任意一个节点），返回结果为复制后复杂链表的head。
（注意，输出结果中请不要返回参数中的节点引用，否则判题程序会直接返回空）

思路一：Python作弊法。【深拷贝、浅拷贝相关知识点】

In [ ]:
class RandomListNode:
    def __init__(self, x):
        self.label = x
        self.next = None
        self.random = None

class Solution:
    # 返回 RandomListNode
    def Clone(self, pHead):
        # write code here
        import copy
        return copy.deepcopy(pHead)

思路二：把复制过程分成两步：

- 第一步复制原始链表上的每个节点，然后用.next连接起来；

- 第二步是设置每个节点的random指针。假设原始链表的某个节点N的random
指向节点S，由于S可能出现在N的前面，也可能出现在N的后面，要确定S的位置，需要从原始链表的头节点开始找。如果从原始链表的头节点开始沿着.next经过
s步找到S，那么在复制链表上节点N'的random指针指向的S'距头节点的距离也是s。这种方法时间复杂度是O(n^2)。

In [ ]:
class Solution2:
    # 返回 RandomListNode
    def Clone(self, pHead):
        # write code here
        if pHead is None:
            return None
        # Step 1
        newHead = self.CloneNodes(pHead)
        # Step 2
        self.ConnectRandomNodes(pHead, newHead)
        return newHead
        
    def CloneNodes(self, pHead):
        head = pHead
        newHead = RandomListNode(pHead.label)
        clonedNode = newHead
        while head.next:
            clonedNode.next = RandomListNode(head.next.label)
            clonedNode = clonedNode.next
            head = head.next
        return newHead
    
    def ConnectRandomNodes(self, pHead, newHead):
        node = pHead  # node of original linked list
        cloned_node = newHead  # node of cloned node
        while node:
            curr_node = pHead
            cloned_curr_node = newHead
            while curr_node != node.random:
                curr_node = curr_node.next
                cloned_curr_node =  cloned_curr_node.next
            cloned_node.random = cloned_curr_node
            node = node.next
            cloned_node = cloned_node.next

思路三：上述方法的时间主要花在定位节点的random指针上，从这方面进行优化。还是分为两步：

- 第一步还是复制原始链表上的每个节点N，然后将这些节点用next连接起来。同时，把<N. N'>的配对信息放到一个哈希表中；

- 第二步还是设置每个节点的random属性。如果在原始链表中节点N的random指向节点S，那么在复制链表中，对应的N'应指向S'。由于有了哈希表，我们可以用O(1)的时间根据S找到S'。

这种方法相当于用空间换时间。对于有n个节点的链表，需要一个大小为O(n)的哈希表，也就是说我们以O(n)的空间消耗把时间复杂度由O(n^2)降低到O(n)。

In [ ]:
# 自己的测试用例测试通过，牛客网上测试未通过，暂时没找到bug

class Solution3:
    # 返回 RandomListNode
    def Clone(self, pHead):
        # write code here
        if pHead is None:
            return None
        # Step 1
        newHead, dict_random = self.CloneNodes(pHead)
        # Step 2
        self.ConnectRandomNodes(pHead, newHead, dict_random)
        return newHead
    
    def CloneNodes(self, pHead):
        dict_random = {}
        node = pHead
        newHead = RandomListNode(pHead.label)
        clonedNode = newHead
        while node.next:  # 用node.next当边界条件的时候要注意最后一个节点是否被遍历到
            clonedNode.next = RandomListNode(node.next.label)
            dict_random[id(node)] = clonedNode
            clonedNode = clonedNode.next
            node = node.next
        dict_random[id(node)] = clonedNode
        return newHead, dict_random
    
    def ConnectRandomNodes(self, pHead, new_head, dict_random):
        node = pHead  # node of original linked list
        cloned_node = new_head  # node of cloned linked list
        while node:
            if node.random is None:
                cloned_node.random = None
            else:
                cloned_node.random = dict_random[id(node.random)]
            node = node.next
            cloned_node = cloned_node.next

思路四：不使用辅助空间的情况下实现O(n)的时间效率。分为三步：

- 第一步：根据原始链表的每个节点N创建对应的N'，这一次，我们把N'接在N的后面。

- 第二步：设置复制出来的节点的random指针。假设原始链表上的N.random指向S，那么N'是N的next指向的节点，同样S'也是S的next指针指向的节点。

- 第三步：把这个长链表拆成两个链表，把偶数位置的节点用next连接起来即为复制出来的链表

【注意】复制过程中不能改变原链表！否则牛客网测试用例不通过。

In [ ]:
class Solution4:
    # 返回 RandomListNode
    def Clone(self, pHead):
        # write code here
        if pHead is None:
            return None
        # Step 1
        self.CloneNodes(pHead)
        # Step 2
        self.ConnectRandomNodes(pHead)
        # step 3
        return self.SplitRandomNodes(pHead)

    def CloneNodes(self, pHead):
        head = pHead
        # clonedNode = RandomListNode(head.label)
        while head:
            ori_next = head.next
            head.next = RandomListNode(head.label)
            head = head.next
            head.next = ori_next
            head = head.next

    def ConnectRandomNodes(self, pHead):
        node = pHead  # node of original linked list
        while node:
            cloned_node = node.next
            if node.random is not None:
                cloned_node.random = node.random.next
            node = cloned_node.next

    def SplitRandomNodes(self, pHead):
        node = pHead
        newHead = node.next
        clonedNode = newHead
        node.next = clonedNode.next
        node = node.next  # 从第三个节点开始
        while node:
            clonedNode.next = node.next
            clonedNode = clonedNode.next
            node.next = clonedNode.next
            node = node.next
        return newHead

~~思路五：递归法~~

存在问题：复制后的链表的random指向的还是原链表中的节点，而不是新链表中的节点。

In [ ]:
class Solution5:
    # 返回 RandomListNode
    def Clone(self, pHead):
        # write code here
        if  pHead is None:
            return None
        newNode = RandomListNode(pHead.label)
        newNode.random = pHead.random
        newNode.next=self.Clone(pHead.next)
        return newNode

测试用例

In [ ]:
n0 = RandomListNode(1)
n1 = RandomListNode(2)
n2 = RandomListNode(3)
n3 = RandomListNode(4)
n4 = RandomListNode(5)

n0.next = n1
n0.random = None
n1.next = n2
n1.random = n0
n2.next = n3
n2.random = None
n3.next = n4
n3.random = n2
n4.random = n3
s = Solution()
cloned = s.Clone(n0)

print("=====Original=====")
node = n0
while node:
    if node.random:
        print("label:", node.label, "random:", node.random.label, "random_address:", node.random)
    else:
        print("label:", node.label, "random:", None, "random_address:", node.random)
    node = node.next

print("=====Cloned=====")
while cloned:
    if cloned.random:
        print("label:", cloned.label, "random:", cloned.random.label, "random_address:", cloned.random)
    else:
        print("label:", cloned.label, "random:", None, "random_address:", cloned.random)
    cloned = cloned.next